<div  style="background-color:white;"><h1 style="text-align:center; color:purple;">Détection de sentiments avec un RNN
</h1>

<p style="text-align:center; color:black"><strong>Préparé  Par :</strong> Hafsa Zian | <strong>Filière :</strong> Machine Learning & IA | <strong>Encadré Par :</strong> M. Harchli Fidae</p>

<hr></div>

<center><h2> 📚 Sommaire<h2> </center>

1. <a href="#intro">Introduction</a><br>

2. <a href="#import">Importation des bibliothèques nécessaires</a><br>

3. <a href="#donnees">Données d'apprentissage</a><br>

4. <a href="#pretraitement">Prétraitement des données</a><br>
 a. <a href="#tokenisation">Tokenisation</a><br>
 b. <a href="#vocabulaire">Création du vocabulaire</a><br>
 c. <a href="#remp">Remplacement chaque mot par son indice dans ce vocabulaire</a><br>
 d. <a href="#raison">Raison d'ajout un jeton pad</a><br>

5. <a href="#encodage">Encodage</a><br> 
    a. <a href="#encode">Encodez chaque phrase sous forme de séquence d’indices </a><br>
    b. <a href="#padding"> Padding sur les séquences </a><br>

6. <a href="#modele">Construction du modèle RNN</a><br>

7. <a href="#entrainement">Entraînement du modèle</a><br>

8. <a href="#prediction">Prédiction</a><br>

9. <a href="#qst">réponses aux questions théoriques</a><br>


### 📌 1. Introduction <span id="intro"></span>

Dans ce travail pratique, nous allons implémenter un modèle de Réseau de Neurones Récurrent (RNN) pour la détection de sentiments à partir de phrases textuelles. L'objectif est de classer chaque phrase comme exprimant un sentiment positif (étiquette 1) ou négatif (étiquette 0), en se basant sur le contenu des mots et leur ordre.


#### 🎯 Objectif 

Construire, entraîner et tester un modèle RNN capable de comprendre et de généraliser les relations entre les mots d'une phrase et le sentiment qu'elle exprime, en utilisant un petit jeu de données étiqueté.


### 🧰 2. Importation des bibliothèques nécessaires <span id="import"></span>

In [1]:
import re  # Pour le nettoyage des phrases (tokenisation simple)
from collections import Counter  # Pour compter les mots
import torch  # PyTorch pour le modèle et les tenseurs
import torch.nn as nn  # Pour les couches du réseau de neurones
from keras.preprocessing.sequence import pad_sequences  # Pour le padding des séquences

### 📊 3. Données d'apprentissage <span id="donnees"></span>

In [2]:
# Phrases d'entraînement
phrases = [
    "Je suis très content",
    "C'était une belle journée",
    "Je suis déçu",
    "C'était horrible",
    "J'adore ce film",
    "Je déteste ce livre"
]

# Labels correspondants (1 = positif, 0 = négatif)
labels = [1, 1, 0, 0, 1, 0]

### 🧹 4. Prétraitement des données <span id="pretraitement"></span>

#### a. Tokenisation  <span id="tokenisation"></span> 

In [4]:
def tokenize(sentence):
    sentence = re.sub(r"[^\w\s]", "", sentence).lower() #pour supprimer ponctuation+miniscule
    return sentence.split()

tokenized_sentences = [tokenize(sent) for sent in phrases] # a chaque phrase
print("Les phrases tokenisées sont : \n", tokenized_sentences)

Les phrases tokenisées sont : 
 [['je', 'suis', 'très', 'content'], ['cétait', 'une', 'belle', 'journée'], ['je', 'suis', 'déçu'], ['cétait', 'horrible'], ['jadore', 'ce', 'film'], ['je', 'déteste', 'ce', 'livre']]


#### b. Vocabulaire contenant tous les mots distincts du corpus  <span id="vocabulaire"></span> 

In [7]:
all_words = [word for sentence in tokenized_sentences for word in sentence] 

vocab = list(set(all_words))

print("Le vocabulaire :", vocab)

Le vocabulaire : ['déteste', 'je', 'ce', 'suis', 'belle', 'cétait', 'très', 'une', 'journée', 'jadore', 'film', 'déçu', 'livre', 'horrible', 'content']


#### c. Remplacement chaque mot par son indice dans ce vocabulaire <span id="remp"></span>  

In [8]:
word_to_idx = {word: idx for idx, word in enumerate(vocab)}
print("Vocabulaire :", vocab)
print("Dictionnaire :", word_to_idx)

Vocabulaire : ['déteste', 'je', 'ce', 'suis', 'belle', 'cétait', 'très', 'une', 'journée', 'jadore', 'film', 'déçu', 'livre', 'horrible', 'content']
Dictionnaire mot → indice : {'déteste': 0, 'je': 1, 'ce': 2, 'suis': 3, 'belle': 4, 'cétait': 5, 'très': 6, 'une': 7, 'journée': 8, 'jadore': 9, 'film': 10, 'déçu': 11, 'livre': 12, 'horrible': 13, 'content': 14}


#### d. Raison d'ajout un jeton pad <span id="raison"></span>  

In [9]:
vocab = ["<pad>"] + list(set(all_words))
word_to_idx = {word: idx for idx, word in enumerate(vocab)}

print("Vocabulaire :", vocab)
print("Dictionnaire :", word_to_idx)

Vocabulaire : ['<pad>', 'déteste', 'je', 'ce', 'suis', 'belle', 'cétait', 'très', 'une', 'journée', 'jadore', 'film', 'déçu', 'livre', 'horrible', 'content']
Dictionnaire mot → indice : {'<pad>': 0, 'déteste': 1, 'je': 2, 'ce': 3, 'suis': 4, 'belle': 5, 'cétait': 6, 'très': 7, 'une': 8, 'journée': 9, 'jadore': 10, 'film': 11, 'déçu': 12, 'livre': 13, 'horrible': 14, 'content': 15}


❓ 
Le jeton 'pad' (pour "padding ") est ajouté au vocabulaire pour permettre le remplissage des séquences de mots , afin qu’elles aient toutes la même longueur .

### 🧮 5. Encodage <span id="encodage"></span>

#### a. Encodez chaque phrase sous forme de séquence d’indices <span id="encode"></span>  


In [11]:
encoded_sentences = []
for sentence in tokenized_sentences:
    encoded_sentence = [word_to_idx[word] for word in sentence if word in word_to_idx]
    encoded_sentences.append(encoded_sentence)
    

#affichage
print("Phrases Encodee :\n",encoded_sentences)

Phrases Encodee :
 [[2, 4, 7, 15], [6, 8, 5, 9], [2, 4, 12], [6, 14], [10, 3, 11], [2, 1, 3, 13]]


#### b. Padding sur les séquences <span id="padding"></span>  

In [12]:
max_length = max(len(seq) for seq in encoded_sentences)
padded_sentences = pad_sequences(encoded_sentences, maxlen=max_length, padding='post', value=word_to_idx["<pad>"])

print("Phrases apres padding :\n", padded_sentences)

Phrases apres padding :
 [[ 2  4  7 15]
 [ 6  8  5  9]
 [ 2  4 12  0]
 [ 6 14  0  0]
 [10  3 11  0]
 [ 2  1  3 13]]


### 🤖 6. Construction du modèle RNN <span id="modele"></span>

In [23]:
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=10, hidden_dim=16):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0) #Une couche Embedding
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True) #Une couche RNN (LSTM)
        self.fc = nn.Linear(hidden_dim, 1) #Une couche linéaire de sortie 
        self.sigmoid = nn.Sigmoid() #+ sigmoïde

    def forward(self, x):
        x = self.embedding(x)
        out, (hidden, cell) = self.lstm(x) 
        out = self.fc(hidden[-1])
        return self.sigmoid(out)   

### 🚀 7. Entraînement du modèle<span id="entrainement"></span>

In [24]:
X_train = torch.tensor(padded_sentences, dtype=torch.long)
y_train = torch.tensor(labels, dtype=torch.float).view(-1, 1)

model = SentimentLSTM(len(vocab))
criterion = nn.BCELoss() #a) une fonction de perte adaptée à la classification binaire.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Entraînement
for epoch in range(500): #b) Entraînez le modèle sur plusieurs époques
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 50 == 0: 
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}") #c) Affichez l’évolution de la perte

Epoch 0, Loss: 0.7144
Epoch 50, Loss: 0.0038
Epoch 100, Loss: 0.0015
Epoch 150, Loss: 0.0010
Epoch 200, Loss: 0.0007
Epoch 250, Loss: 0.0005
Epoch 300, Loss: 0.0004
Epoch 350, Loss: 0.0003
Epoch 400, Loss: 0.0003
Epoch 450, Loss: 0.0002


In [29]:
class SentimentGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim=10, hidden_dim=16):
        super(SentimentGRU, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=word_to_idx["<pad>"])
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        out, hidden = self.gru(x)
        out = self.fc(hidden[-1])
        return self.sigmoid(out)

# Initialisation du modèle
model2 = SentimentGRU(vocab_size=len(vocab))


In [30]:
for epoch in range(100): 
    model2.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.0002
Epoch 10, Loss: 0.0002
Epoch 20, Loss: 0.0002
Epoch 30, Loss: 0.0002
Epoch 40, Loss: 0.0002
Epoch 50, Loss: 0.0002
Epoch 60, Loss: 0.0002
Epoch 70, Loss: 0.0002
Epoch 80, Loss: 0.0002
Epoch 90, Loss: 0.0002


### 🧪 8. Prédiction <span id="prediction"></span>

In [27]:
def predict(sentence): #a) Implémentez une fonction predict(phrase) qui retourne :
                        #"Positif" si le modèle prédit une probabilité > 0.5
                        #"Négatif" sinon
    model.eval()
    tokens = tokenize(sentence)
    encoded = [word_to_idx.get(word, 0) for word in tokens]
    padded = pad_sequences([encoded], maxlen=padded_sentences.shape[1], padding='post')
    tensor_input = torch.tensor(padded, dtype=torch.long)
    prob = model(tensor_input).item()
    return "Positif" if prob > 0.5 else "Négatif"


In [28]:
#b) Test
print(predict("je suis heureux"))   
print(predict("je suis triste"))

Négatif
Négatif


In [ ]:
def predict(sentence): #par gru
    model2.eval()
    tokens = tokenize(sentence)
    encoded = [word_to_idx.get(word, word_to_idx["<pad>"]) for word in tokens]
    padded = pad_sequences([encoded], maxlen=max_length, padding='post', value=word_to_idx["<pad>"])
    tensor_input = torch.tensor(padded, dtype=torch.long)
    prob = model2(tensor_input).item()
    print(f"Probabilité: {prob:.4f}")
    return "Positif" if prob > 0.5 else "Négatif"

In [32]:
print(predict("je suis heureux"))
print(predict("je suis triste"))

Probabilité: 0.4594
Négatif
Probabilité: 0.4594
Négatif


### ❓ 9. Réponses aux questions du TP <span id="qst"></span>

##### 9.1. Pourquoi un RNN est-il utile ici, au lieu d’un réseau classique ?

- Car le GRU prend en compte l’ordre des mots dans une phrase, ce qui est essentiel pour bien detecter le sentiment ou bien le sens du phrase par contre un réseau classique ne prend pas en compte l’ordre des mots.


##### 9.2. Quels sont les inconvénients d’un RNN simple ?

- Problèmes de gradient (disparition/exploration), et a du mal à mémoriser des informations importantes sur de longues séquences.

##### 9.3. Que se passerait-il si on avait des phrases beaucoup plus longues ?

- Un RNN simple aurait encore plus de mal, il faudrait utiliser LSTM ou Transformer.